In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from performance_calculation import *

In [20]:
# Small function to convert the strings to arrays.
def conv2arr(array):
    string_arr = array.split(', ')
    if string_arr[-1][-1] == ',':
        string_arr[-1] = string_arr[-1][0]
    return np.array([[int(numeric_string) for numeric_string in string_arr]])

[[ 2  2  8  9  9  8 -1  5 -1  9  6  9  4  2  0  5  1 -1  3  6  0  0  8  6
   6  8  8  0  9  1  8  1  5  6  9  1  6  1  3  6  0  0 -1  6  1 -1  7 -1
   2  5  7 -1  1  9  5  8  3  0  1  2  3  4  6]]


In [10]:
experiments = pd.read_csv('experiments.csv')
experiments['encoding input'][0]

'2, 2, 8, 9, 9, 8, -1, 5, -1, 9, 6, 9, 4, 2, 0, 5, 1, -1, 3, 6, 0, 0, 8, 6, 6, 8, 8, 0, 9, 1, 8, 1, 5, 6, 9, 1, 6, 1, 3, 6, 0, 0, -1, 6, 1, -1, 7, -1, 2, 5, 7, -1, 1, 9, 5, 8, 3, 0, 1, 2, 3, 4, 6,'

In [22]:
# Plot the difference in ITR and accuracy for different time delays.

# Get all t trials
t_trials = experiments[experiments['trial name'].str.contains('t')]
t_trials_a = t_trials[t_trials['subject'] == 'Ahmed']
t_trials_a_decoding_itr = t_trials_a.apply(lambda x: calc_itr(conv2arr(x['encoding input']), conv2arr(x['decoding output']), x['time (s)']), axis=1)
t_trials_n = t_trials[t_trials['subject'] == 'Nathan']

# Add the trials to the figure.
fig, ax1 = plt.subplots()
ax1.set_xlabel('Time delay (ms)')
ax1.set_ylabel('Information Transfer Rate (bpm)')
ax1.plot(t_trials_a['Time delay (ms)'][:-1], t_trials_a['itr'][:-1], color='blue', label='Ahmed ITR')
ax1.plot(t_trials_n['Time delay (ms)'][:-1], t_trials_n['itr'][:-1], color='green', label='Nathan ITR')
ax1.scatter(t_trials_a['Time delay (ms)'].iloc[-1], t_trials_a['itr'].iloc[-1], color='blue')
ax1.scatter(t_trials_n['Time delay (ms)'].iloc[-1], t_trials_n['itr'].iloc[-1], color='green')
ax1.legend(loc='lower right')

ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy')
ax2.plot(t_trials_a['Time delay (ms)'][:-1], t_trials_a['accuracy'][:-1], color='blue', label='Ahmed accuracy', linestyle='--')
ax2.plot(t_trials_n['Time delay (ms)'][:-1], t_trials_n['accuracy'][:-1], color='green', label='Nathan accuracy', linestyle='--')
ax2.scatter(t_trials_a['Time delay (ms)'].iloc[-1], t_trials_a['accuracy'].iloc[-1], color='blue', marker='x')
ax2.scatter(t_trials_n['Time delay (ms)'].iloc[-1], t_trials_n['accuracy'].iloc[-1], color='green', marker='x')
ax2.legend(loc='upper left')

plt.title('Tuning the vibration delay')
fig.tight_layout()
plt.show()

ValueError: operands could not be broadcast together with shapes (1,49) (1,50) 